In [ ]:
SELECT PurchaseOrderID, SupplierID, OrderDate
FROM Purchasing.PurchaseOrders
WHERE SupplierID IN (SELECT CustomerID
                     FROM Sales.CustomerTransactions
                     WHERE IsFinalized = 1);

<span style="color: #008000;">The inner query finds all the customer Ids with finalized transactions and</span>

<span style="color: #008000;">the outer query retrives purchases orders where the supplier ID is in the list of CustomerIds form the innner query</span>

In [ ]:
SELECT CustomerTransactionID, CustomerID, TransactionDate, TransactionAmount
FROM Sales.CustomerTransactions AS CT1
WHERE EXISTS (SELECT 1
              FROM Sales.CustomerTransactions AS CT2
              WHERE CT1.CustomerID = CT2.CustomerID
              AND CT1.TransactionDate = CT2.TransactionDate
              AND CT1.CustomerTransactionID <> CT2.CustomerTransactionID);

<span style="color: #008000;">This correlated subquery finds all the customers who made more than one tranaction on one day</span>

<span style="color: #008000;">the subquery checks whether there exists another transaction in the sub query</span>

<span style="color: #008000;">by the same customer in the outer query by comparing CT1 and CT2</span>

In [ ]:
SELECT CustomerTransactionID, CustomerID, TransactionAmount
FROM Sales.CustomerTransactions
WHERE TransactionAmount > (SELECT AVG(TransactionAmount) FROM Sales.CustomerTransactions);

<span style="color: #008000;">This subquery returns the transaction ammounts that are above the average</span>

In [ ]:
SELECT PurchaseOrderID, SupplierID, OrderDate
FROM [Purchasing].[PurchaseOrders]
WHERE OrderDate > (SELECT MAX(TransactionDate) 
                   FROM [Sales].[CustomerTransactions]);

<span style="color: #008000;">The inner query gets the latest tranaction date and the outer</span>

<span style="color: #008000;">query gets all purchase orders with an OrderDate later than the max tranaction date</span>

In [ ]:
SELECT PurchaseOrderID, SupplierID, DeliveryMethodID
FROM Purchasing.PurchaseOrders PO1
WHERE DeliveryMethodID = (SELECT TOP 1 DeliveryMethodID 
                          FROM Purchasing.PurchaseOrders PO2
                          WHERE PO1.SupplierID = PO2.SupplierID);

<span style="color: #008000;">Gets the delivery methodID of the same supplier for the inner query</span>

<span style="color: #008000;">does this by referencing PO1.SupplierID from outer query</span>

In [ ]:
SELECT StockItemID, TotalStock, LastStocktakeQuantity
FROM (
    SELECT StockItemID, SUM(QuantityOnHand) AS TotalStock, LastStocktakeQuantity
    FROM [Warehouse].[StockItemHoldings]
    WHERE QuantityOnHand > 50
    GROUP BY StockItemID, LastStocktakeQuantity
) AS StockSummary;

<span style="color: #008000;">Gets the total stock on hand with a stock level above 50</span>

In [ ]:
SELECT CustomerID, SUM(TaxAmount) AS TotalTax
FROM [Sales].[CustomerTransactions]
GROUP BY CustomerID
HAVING SUM(TaxAmount) > 500;

<span style="color: #008000;">Finds customers whose total tax amount for all transaction is above 500 dollars</span>

In [ ]:
WITH CustomerPaymentMethods AS (
    SELECT CustomerID, COUNT(DISTINCT PaymentMethodID) AS PaymentMethodCount
    FROM [Sales].[CustomerTransactions]
    GROUP BY CustomerID
)
SELECT CustomerID
FROM CustomerPaymentMethods
WHERE PaymentMethodCount > 1;

<span style="color: #008000;">Gets all customers who have used more than one payment</span>

In [ ]:
WITH UnfinalizedTransactions AS (
    SELECT InvoiceID, CustomerTransactionID, IsFinalized
    FROM [Sales].[CustomerTransactions]
    WHERE IsFinalized = 0
)
SELECT UT.InvoiceID, UT.CustomerTransactionID
FROM UnfinalizedTransactions UT
JOIN [Sales].[Invoices] I 
    ON UT.InvoiceID = I.InvoiceID;

<span style="color: #008000;">Gets all invoices of customer tranactions but are not finalized</span>

In [ ]:
WITH CustomerTransactionCounts AS (
    SELECT CustomerID, COUNT(*) AS TransactionCount, AVG(TransactionAmount) AS AvgTransactionAmount
    FROM [Sales].[CustomerTransactions]
    WHERE IsFinalized = 1
    GROUP BY CustomerID
)
SELECT CustomerID, TransactionCount, AvgTransactionAmount
FROM CustomerTransactionCounts
WHERE TransactionCount > 3
ORDER BY AvgTransactionAmount DESC;

<span style="color: #008000;">This query gets us the CTE or CustomerTransactionCount and retrieves each</span>

<span style="color: #008000;">customerID with finalized accounts and aver transaction amount over 3</span>